# `Python to MySQL: Hiccups you will encounter`

# <font color=red>Mr Fugu Data Science</font>

# `Outcome:`

+ `Problems with Date-Time`
+ `Handling NaN`
+ `Schema Considerations`
+ `Dot Notation`

In [ ]:
import pandas as pd # dataframe stuff
import json # 
from collections import defaultdict # dictionary where values are a list
import numpy as np # formatting data with Nan
from datetime import date
import datetime

import collections # This is used to help find duplicate values later

In [ ]:
db_config = read_db_config() # user credentials are mine, you will need to change this line

conn = MySQLConnection(**read_db_config()) # establish connection

mycursor = conn.cursor() # cursor, allows us to do queries,send/retrieve data

# `If you want user credential setup go here:`

https://github.com/MrFuguDataScience/SendDataFrom-R-Python-to-SQL-PSQL-Mongo/blob/master/Python_to_mysql_configFiles.ipynb

IF YOU DO NOT HAVE CREDENTIALS OR IT DOESN'T MATTER TO YOU TRY THIS:

`
cnx = mysql.connector.connect(user='bambam', password='password',
                              host='127.0.0.1',
                              database='cowboy_doodah')
cnx.close()
`

+ Remember: replace, user, password and database with your information!

In [ ]:
# Setup MySQL for us to send data and create a new DB

import mysql.connector as mysql # establish connection to mysql 

from mysql.connector import MySQLConnection # using Class for function call

import pandas as pd  # Create Dataframe

from python_to_myql_config import read_db_config # calling config files, (you need to make this) 
# stored user credentials I have, or use the block of code below

import csv